In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import time

# Definizione del modello
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # Primo strato
        self.dropout1 = nn.Dropout(0.2)        # Dropout
        self.fc2 = nn.Linear(64, 32)           # Secondo strato
        self.dropout2 = nn.Dropout(0.2)        # Secondo Dropout
        self.fc3 = nn.Linear(32, 2)            # Strato di output (2 classi)

    def forward(self, x):
        x = torch.relu(self.fc1(x))            # Funzione di attivazione ReLU
        x = self.dropout1(x)                    # Dropout
        x = torch.relu(self.fc2(x))            # Funzione di attivazione ReLU
        x = self.dropout2(x)                    # Secondo Dropout
        x = self.fc3(x)                         # Output senza softmax
        return x

# Funzione per addestrare il modello
def train_model(optimizer, criterion, model, X_train, y_train, X_test, y_test, num_epochs=200):
    final_train_loss = None
    final_test_loss = None
    for epoch in range(num_epochs):
        model.train()  # Imposta il modello in modalità di addestramento
        optimizer.zero_grad()  # Azzerare i gradienti

        # Forward pass
        outputs = model(X_train)
        loss = criterion(outputs, y_train)

        # Backward pass e ottimizzazione
        loss.backward()
        optimizer.step()

        # Valutazione sul set di test
        model.eval()  # Imposta il modello in modalità di valutazione
        with torch.no_grad():
            test_outputs = model(X_test)
            test_loss = criterion(test_outputs, y_test)
            
            # Calcolo dell'accuratezza
            _, predicted = torch.max(test_outputs, 1)
            accuracy = (predicted == y_test).sum().item() / y_test.size(0)

        final_train_loss = loss.item()
        final_test_loss = test_loss.item()

        print(f'Epoch {epoch + 1}, Train Loss: {loss.item()}, Test Loss: {test_loss.item()}, Accuracy: {accuracy:.4f}')

    return accuracy, final_train_loss, final_test_loss

# Caricamento dei dati
path1 = 'dataset/train_data.csv'
train_data = pd.read_csv(path1)

path2 = 'dataset/test_data.csv'
test_data = pd.read_csv(path2)

# Supponiamo che i dati siano già pre-processati
X_train = torch.tensor(train_data.drop(columns='income').values, dtype=torch.float32)
y_train = torch.tensor(train_data['income'].values, dtype=torch.long)
X_test = torch.tensor(test_data.drop(columns='income').values, dtype=torch.float32)
y_test = torch.tensor(test_data['income'].values, dtype=torch.long)

# Elenco degli ottimizzatori da provare
optimizers = {
    "SGD": optim.SGD,
    "SGD + Momentum": lambda params: optim.SGD(params, lr=0.01, momentum=0.9),
    "Adam": optim.Adam,
    "Adagrad": optim.Adagrad,
    "Adamax": optim.Adamax,
    "RMSprop": optim.RMSprop,
    "Adadelta": optim.Adadelta,
    "Nadam": optim.NAdam
}

# Iterare sugli ottimizzatori
for name in optimizers.keys():
    print(f"\nTraining with optimizer: {name}")
    
    # Inizializza un nuovo modello per ogni ottimizzatore
    model = SimpleNN(input_size=X_train.shape[1])  # Inizializza un nuovo modello
    criterion = nn.CrossEntropyLoss()  # Funzione di perdita

    # Inizializza l'ottimizzatore con i parametri del modello
    if name == "SGD + Momentum":
        optimizer_instance = optimizers[name](model.parameters())  # Usa la lambda
    else:
        optimizer_instance = optimizers[name](model.parameters(), lr=0.001)

    # Misura il tempo di addestramento
    start_time = time.time()
    
    # Addestra il modello e ottieni i risultati finali
    final_accuracy, final_train_loss, final_test_loss = train_model(
        optimizer_instance, criterion, model, X_train, y_train, X_test, y_test, num_epochs=200
    )
    
    end_time = time.time()
    training_time = end_time - start_time
    
    # Stampa i risultati finali
    print(f'Optimizer: {name}')
    print(f'Final Test Accuracy: {final_accuracy:.4f}')
    print(f'Training Time: {training_time:.2f} seconds')
    print(f'Final Train Loss: {final_train_loss:.4f}')
    print(f'Final Test Loss: {final_test_loss:.4f}')

print("Training complete.")



Training with optimizer: SGD
Epoch 1, Train Loss: 754.5484008789062, Test Loss: 27141.7890625, Accuracy: 0.7656
Epoch 2, Train Loss: 29574.125, Test Loss: 26330132.0, Accuracy: 0.2344
Epoch 3, Train Loss: 25857896.0, Test Loss: 111624.4375, Accuracy: 0.7656
Epoch 4, Train Loss: 114204.09375, Test Loss: 324880640.0, Accuracy: 0.2344
Epoch 5, Train Loss: 324413248.0, Test Loss: 982643621494784.0, Accuracy: 0.7656
Epoch 6, Train Loss: 1013837700530176.0, Test Loss: 125887.078125, Accuracy: 0.2344
Epoch 7, Train Loss: 124373.8203125, Test Loss: 24728.712890625, Accuracy: 0.7656
Epoch 8, Train Loss: 25348.375, Test Loss: 16537.44140625, Accuracy: 0.7656
Epoch 9, Train Loss: 16884.15234375, Test Loss: 8378.89453125, Accuracy: 0.7656
Epoch 10, Train Loss: 8613.9111328125, Test Loss: 163.7344970703125, Accuracy: 0.7656
Epoch 11, Train Loss: 167.9480438232422, Test Loss: 0.6720032095909119, Accuracy: 0.7656
Epoch 12, Train Loss: 0.6725071668624878, Test Loss: 0.6718862652778625, Accuracy: 0.76

Epoch 95, Train Loss: 0.6634199023246765, Test Loss: 0.6625717878341675, Accuracy: 0.7656
Epoch 96, Train Loss: 0.663314938545227, Test Loss: 0.6624642610549927, Accuracy: 0.7656
Epoch 97, Train Loss: 0.6632100939750671, Test Loss: 0.6623567342758179, Accuracy: 0.7656
Epoch 98, Train Loss: 0.6631053686141968, Test Loss: 0.6622491478919983, Accuracy: 0.7656
Epoch 99, Train Loss: 0.6630005836486816, Test Loss: 0.6621420979499817, Accuracy: 0.7656
Epoch 100, Train Loss: 0.6628962755203247, Test Loss: 0.662034809589386, Accuracy: 0.7656
Epoch 101, Train Loss: 0.6627916693687439, Test Loss: 0.6619276404380798, Accuracy: 0.7656
Epoch 102, Train Loss: 0.662687361240387, Test Loss: 0.661820650100708, Accuracy: 0.7656
Epoch 103, Train Loss: 0.6625832319259644, Test Loss: 0.6617136597633362, Accuracy: 0.7656
Epoch 104, Train Loss: 0.6624789834022522, Test Loss: 0.6616067886352539, Accuracy: 0.7656
Epoch 105, Train Loss: 0.6623749136924744, Test Loss: 0.6615001559257507, Accuracy: 0.7656
Epoch 10

Epoch 188, Train Loss: 0.6540919542312622, Test Loss: 0.6529999375343323, Accuracy: 0.7656
Epoch 189, Train Loss: 0.6539962887763977, Test Loss: 0.6529015302658081, Accuracy: 0.7656
Epoch 190, Train Loss: 0.6539006233215332, Test Loss: 0.6528035402297974, Accuracy: 0.7656
Epoch 191, Train Loss: 0.6538052558898926, Test Loss: 0.6527054905891418, Accuracy: 0.7656
Epoch 192, Train Loss: 0.6537097692489624, Test Loss: 0.6526073813438416, Accuracy: 0.7656
Epoch 193, Train Loss: 0.6536142826080322, Test Loss: 0.6525096893310547, Accuracy: 0.7656
Epoch 194, Train Loss: 0.6535192728042603, Test Loss: 0.652411937713623, Accuracy: 0.7656
Epoch 195, Train Loss: 0.6534242033958435, Test Loss: 0.6523140668869019, Accuracy: 0.7656
Epoch 196, Train Loss: 0.6533290147781372, Test Loss: 0.6522165536880493, Accuracy: 0.7656
Epoch 197, Train Loss: 0.6532341837882996, Test Loss: 0.6521191000938416, Accuracy: 0.7656
Epoch 198, Train Loss: 0.6531394124031067, Test Loss: 0.6520217061042786, Accuracy: 0.7656


Epoch 81, Train Loss: 0.5516843199729919, Test Loss: 0.5445771813392639, Accuracy: 0.7656
Epoch 82, Train Loss: 0.551684558391571, Test Loss: 0.5445771217346191, Accuracy: 0.7656
Epoch 83, Train Loss: 0.5516846179962158, Test Loss: 0.5445772409439087, Accuracy: 0.7656
Epoch 84, Train Loss: 0.5516844391822815, Test Loss: 0.5445774793624878, Accuracy: 0.7656
Epoch 85, Train Loss: 0.5516840815544128, Test Loss: 0.5445781350135803, Accuracy: 0.7656
Epoch 86, Train Loss: 0.5516836047172546, Test Loss: 0.5445786714553833, Accuracy: 0.7656
Epoch 87, Train Loss: 0.5516828298568726, Test Loss: 0.5445793867111206, Accuracy: 0.7656
Epoch 88, Train Loss: 0.5516819357872009, Test Loss: 0.5445802211761475, Accuracy: 0.7656
Epoch 89, Train Loss: 0.5516809821128845, Test Loss: 0.5445811748504639, Accuracy: 0.7656
Epoch 90, Train Loss: 0.5516800880432129, Test Loss: 0.5445823669433594, Accuracy: 0.7656
Epoch 91, Train Loss: 0.5516789555549622, Test Loss: 0.5445835590362549, Accuracy: 0.7656
Epoch 92, T

Epoch 175, Train Loss: 0.5516539812088013, Test Loss: 0.5446588397026062, Accuracy: 0.7656
Epoch 176, Train Loss: 0.551654040813446, Test Loss: 0.5446587204933167, Accuracy: 0.7656
Epoch 177, Train Loss: 0.5516538023948669, Test Loss: 0.5446587204933167, Accuracy: 0.7656
Epoch 178, Train Loss: 0.5516538023948669, Test Loss: 0.5446585416793823, Accuracy: 0.7656
Epoch 179, Train Loss: 0.5516537427902222, Test Loss: 0.5446587204933167, Accuracy: 0.7656
Epoch 180, Train Loss: 0.5516538023948669, Test Loss: 0.5446587204933167, Accuracy: 0.7656
Epoch 181, Train Loss: 0.5516538023948669, Test Loss: 0.544658899307251, Accuracy: 0.7656
Epoch 182, Train Loss: 0.551654040813446, Test Loss: 0.5446587800979614, Accuracy: 0.7656
Epoch 183, Train Loss: 0.5516538023948669, Test Loss: 0.5446587800979614, Accuracy: 0.7656
Epoch 184, Train Loss: 0.5516538619995117, Test Loss: 0.5446587800979614, Accuracy: 0.7656
Epoch 185, Train Loss: 0.5516539812088013, Test Loss: 0.5446587800979614, Accuracy: 0.7656
Ep

...

Epoch 14, Train Loss: 6755.21923828125, Test Loss: 6563.095703125, Accuracy: 0.2344
Epoch 15, Train Loss: 6740.69189453125, Test Loss: 6559.5693359375, Accuracy: 0.2344
Epoch 16, Train Loss: 6738.1318359375, Test Loss: 6556.0224609375, Accuracy: 0.2344
Epoch 17, Train Loss: 6743.0126953125, Test Loss: 6552.46533203125, Accuracy: 0.2344
Epoch 18, Train Loss: 6731.06201171875, Test Loss: 6548.89599609375, Accuracy: 0.2344
Epoch 19, Train Loss: 6720.07275390625, Test Loss: 6545.3095703125, Accuracy: 0.2344
Epoch 20, Train Loss: 6715.7998046875, Test Loss: 6541.7119140625, Accuracy: 0.2344
Epoch 21, Train Loss: 6725.6416015625, Test Loss: 6538.09423828125, Accuracy: 0.2344
Epoch 22, Train Loss: 6719.87060546875, Test Loss: 6534.45703125, Accuracy: 0.2344
Epoch 23, Train Loss: 6707.2119140625, Test Loss: 6530.81396484375, Accuracy: 0.2344
Epoch 24, Train Loss: 6703.30712890625, Test Loss: 6527.158203125, Accuracy: 0.2344
Epoch 25, Train Loss: 6713.77783203125, Test Loss: 6523.49267578125, A

Epoch 111, Train Loss: 6423.755859375, Test Loss: 6190.0400390625, Accuracy: 0.2344
Epoch 112, Train Loss: 6393.76611328125, Test Loss: 6185.7724609375, Accuracy: 0.2344
Epoch 113, Train Loss: 6350.8388671875, Test Loss: 6181.50146484375, Accuracy: 0.2344
Epoch 114, Train Loss: 6385.6591796875, Test Loss: 6177.19921875, Accuracy: 0.2344
Epoch 115, Train Loss: 6374.3173828125, Test Loss: 6172.88916015625, Accuracy: 0.2344
Epoch 116, Train Loss: 6367.00634765625, Test Loss: 6168.5693359375, Accuracy: 0.2344
Epoch 117, Train Loss: 6361.12744140625, Test Loss: 6164.23681640625, Accuracy: 0.2344
Epoch 118, Train Loss: 6352.68017578125, Test Loss: 6159.9072265625, Accuracy: 0.2344
Epoch 119, Train Loss: 6339.98876953125, Test Loss: 6155.564453125, Accuracy: 0.2344
Epoch 120, Train Loss: 6342.76513671875, Test Loss: 6151.21435546875, Accuracy: 0.2344
Epoch 121, Train Loss: 6343.62890625, Test Loss: 6146.8544921875, Accuracy: 0.2344
Epoch 122, Train Loss: 6353.7978515625, Test Loss: 6142.48193

Epoch 7, Train Loss: 569.7258911132812, Test Loss: 412.31256103515625, Accuracy: 0.7659
Epoch 8, Train Loss: 555.1062622070312, Test Loss: 404.96929931640625, Accuracy: 0.7661
Epoch 9, Train Loss: 524.7977294921875, Test Loss: 387.9916076660156, Accuracy: 0.7661
Epoch 10, Train Loss: 498.5072937011719, Test Loss: 370.14813232421875, Accuracy: 0.7664
Epoch 11, Train Loss: 473.72564697265625, Test Loss: 347.9034423828125, Accuracy: 0.7670
Epoch 12, Train Loss: 464.1806945800781, Test Loss: 326.36834716796875, Accuracy: 0.7678
Epoch 13, Train Loss: 434.1370849609375, Test Loss: 298.0254821777344, Accuracy: 0.7680
Epoch 14, Train Loss: 409.2530517578125, Test Loss: 265.67724609375, Accuracy: 0.7682
Epoch 15, Train Loss: 397.9288635253906, Test Loss: 235.9463653564453, Accuracy: 0.7685
Epoch 16, Train Loss: 371.743408203125, Test Loss: 206.45437622070312, Accuracy: 0.7686
Epoch 17, Train Loss: 353.7148132324219, Test Loss: 180.12850952148438, Accuracy: 0.7689
Epoch 18, Train Loss: 334.80041

Epoch 101, Train Loss: 4.1130051612854, Test Loss: 0.6448564529418945, Accuracy: 0.7716
Epoch 102, Train Loss: 4.027729511260986, Test Loss: 0.6442629098892212, Accuracy: 0.7716
Epoch 103, Train Loss: 3.9121716022491455, Test Loss: 0.6435182094573975, Accuracy: 0.7716
Epoch 104, Train Loss: 3.5618269443511963, Test Loss: 0.6429321765899658, Accuracy: 0.7716
Epoch 105, Train Loss: 3.507821559906006, Test Loss: 0.6422054767608643, Accuracy: 0.7716
Epoch 106, Train Loss: 3.6311116218566895, Test Loss: 0.6416236758232117, Accuracy: 0.7716
Epoch 107, Train Loss: 3.442039966583252, Test Loss: 0.640879213809967, Accuracy: 0.7716
Epoch 108, Train Loss: 3.519256830215454, Test Loss: 0.640343427658081, Accuracy: 0.7716
Epoch 109, Train Loss: 3.150158643722534, Test Loss: 0.6396867632865906, Accuracy: 0.7716
Epoch 110, Train Loss: 3.1511237621307373, Test Loss: 0.63905268907547, Accuracy: 0.7716
Epoch 111, Train Loss: 3.2918622493743896, Test Loss: 0.6383917927742004, Accuracy: 0.7716
Epoch 112, 

Epoch 192, Train Loss: 1.360485315322876, Test Loss: 0.6019695401191711, Accuracy: 0.7706
Epoch 193, Train Loss: 1.3770638704299927, Test Loss: 0.6016109585762024, Accuracy: 0.7706
Epoch 194, Train Loss: 1.4377329349517822, Test Loss: 0.6013264060020447, Accuracy: 0.7706
Epoch 195, Train Loss: 1.2669659852981567, Test Loss: 0.6010135412216187, Accuracy: 0.7706
Epoch 196, Train Loss: 1.2972047328948975, Test Loss: 0.6006584167480469, Accuracy: 0.7705
Epoch 197, Train Loss: 1.4086403846740723, Test Loss: 0.6003648042678833, Accuracy: 0.7704
Epoch 198, Train Loss: 1.3189783096313477, Test Loss: 0.6000396609306335, Accuracy: 0.7704
Epoch 199, Train Loss: 1.2999564409255981, Test Loss: 0.5997444987297058, Accuracy: 0.7704
Epoch 200, Train Loss: 1.3630156517028809, Test Loss: 0.5994051694869995, Accuracy: 0.7704
Optimizer: Nadam
Final Test Accuracy: 0.7704
Training Time: 9.87 seconds
Final Train Loss: 1.3630
Final Test Loss: 0.5994
Training complete.
